In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import joblib
from torchvision import transforms
from PIL import Image

# Load YOLOv8 model (trained to detect faces and classify gender: 0=Male, 1=Female)
yolo_model = YOLO("best.pt")  # your YOLOv8 trained weights

# Load your 16-class identity classifier
classifier = joblib.load("face_classifier_16.pkl")  # your identity classifier

# Image transform for classifier
transform = transforms.Compose([
    transforms.Resize((64, 64)),     # adjust to your model's expected input size
    transforms.ToTensor()
])

# Load the input image
image_path = "test.jpg"  # change this to your input image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run YOLO model
results = yolo_model(image_rgb)

# Loop through detected faces
for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    gender_cls = int(box.cls[0])  # 0 = Male, 1 = Female

    # Crop the face
    face_crop = image_rgb[y1:y2, x1:x2]
    if face_crop.size == 0:
        continue

    # Prepare for classifier
    face_pil = Image.fromarray(face_crop)
    input_tensor = transform(face_pil).unsqueeze(0)  # shape: [1, 3, 64, 64]
    input_tensor = input_tensor.view(1, -1)  # flatten for sklearn classifier if needed

    # Predict class (0 to 15)
    predicted_class = classifier.predict(input_tensor)[0]
    label_text = f"Belongs to Class {predicted_class}"

    # Set box color
    color = (0, 255, 0) if gender_cls == 1 else (0, 0, 255)  # Green for female, red for male

    # Draw box and label
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

# Save the final output
cv2.imwrite("output.jpg", image)
print("✅ Final image saved as output.jpg")